# Model Deployment 

##  Setting up the notebook

In [35]:
# importing the necossery liberay 

import sagemaker
from sagemaker import get_execution_role


import os
import boto3
import json
import numpy as np
import tensorflow as tf

from pickle import load

In [36]:
# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
sm_session = sagemaker.Session()

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = get_execution_role()

## Deploy the trained model using 


### reference documentataion & useful links  :

https://docs.aws.amazon.com/sagemaker/latest/dg/neo-deployment-hosting-services-prerequisites.html
    
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html 
    
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html 

https://github.com/aws/sagemaker-python-sdk/blob/6acde94c286087d2d1d991bef28a63a47f53810a/src/sagemaker/predictor.py


In [37]:

s3_model_location = 's3://sagemaker-us-east-1-478270364551/tensorflow-training-2021-02-28-13-12-34-041/output/model.tar.gz'

In [38]:

from sagemaker.tensorflow import TensorFlowModel

 
model = TensorFlowModel(
                         model_data= s3_model_location ,
                         role = role,
                         framework_version='2.1.0',
                         entry_point='inference.py',
                         source_dir='server',
                         sagemaker_session= sm_session,
                        )


In [39]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------------!

In [40]:

predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'tensorflow-inference-2021-03-01-16-42-22-041'

## Test the working 

In [41]:
## setup to invoke endpoint
sm_runtime_client = boto3.client('sagemaker-runtime')

In [42]:

test_features = load(open('/home/ec2-user/SageMaker/Data Preprocessing/Dataset/test_features' ,'rb'))

sample_feature = test_features[list(test_features.keys())[0]][0]

In [43]:
sample_feature.shape

(1000,)

In [44]:
image_list = sample_feature.tolist()
image_json = json.dumps(image_list)

In [45]:
## invoke endpoint for inference
response = sm_runtime_client.invoke_endpoint(
                                            EndpointName=predictor.endpoint,
                                            Body=image_json,
                                            ContentType="application/json",
                                            )


The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [46]:
json.loads(response['Body'].read().decode('utf-8'))

'young girl in pink shirt is playing guitar'

## Delete the endpoint (if not needed)

In [34]:
predictor.delete_endpoint()